In [4]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random
import pandas as pd

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}


def get_news(news_url):
    news_detail = []
    print(news_url)
    req = requests.get(news_url, headers=header)
    soup = BeautifulSoup(req.content, 'html.parser')

    # 제목
    title = soup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    # 날짜 
    pdate = soup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate)

    # 본문
    _text = soup.select('#articleBodyContents')[
        0].get_text().replace('\n', " ")
    text = _text.replace(
        "// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(text.strip())

    # 언론사
    pcompany = soup.select('div.article_footer')[
        0].a.get_text().split()[0].strip()
    news_detail.append(pcompany)

    return news_detail

columns = ['날짜', '신문사', '제목', '내용']
df = pd.DataFrame(columns=columns)

query = '코로나'   # url 인코딩 에러는 encoding parse.quote(query)
s_date = "2020.04.01"
e_date = "2020.04.08"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

while True:
    time.sleep(random.sample(range(3), 1)[0])
    print(page)

    url = "https://search.naver.com/search.naver?where=news&query=" + query + \
        "&sort=1&field=1&ds=" + s_date + "&de=" + e_date + \
        "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + \
        "%2Ca%3A&start=" + str(page)

    req = requests.get(url, headers=header)
    print(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    naver_news = soup.find_all("a", {"class": "info"})
    if naver_news == []:
        break

    for a_tag in naver_news:
        try:
            news_url = a_tag.attrs["href"]
            if news_url.startswith("https://news.naver.com"):
                print(news_url)
                news_detail = get_news(news_url)
                print(news_detail)
                #df = df.append(pd.DataFrame(
                #    [[news_detail[1], news_detail[3], news_detail[0], news_detail[2]]], columns=columns))
        except Exception as e:
            print(e)
            continue
    break
    
    page += 10

1
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.04.01&de=2020.04.08&nso=so%3Ar%2Cp%3Afrom20200401to20200408%2Ca%3A&start=1
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533947
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011533947
['러시아 코로나19 신규 확진자 이틀째 1천명대…"전체 8천672명"(종합)', '2020.04.08.', '"모스크바 발병자 5천841명"…현지 전문가 "6월중순께나 수그러들 것"(모스크바=연합뉴스) 유철종 특파원 = 러시아의 신종 코로나바이러스 감염증(코로나19) 확진자 수가 하루 1천명대 이상의 증가세를 보이면서 누적 확진자가 8천600명을 넘어섰다.     타스 통신 등에 따르면 러시아 정부의 코로나19 유입 및 확산방지 대책본부는 8일(현지시간) "지난 하루 동안 모스크바를 포함한 56개 지역에서 1천175명의 추가 확진자가 나왔다"면서 "전체 누적 확진자가 8천672명(81개 지역)으로 늘었다"고 밝혔다.     수도 모스크바에선 660명의 추가 확진자가 나오면서 전체 감염자가 5천841명으로 증가했다.    모스크바시 대책본부는 "관내 신규 확진자 가운데 85%는 65세 이하"라며 "18세~45세 청장년층 비율이 40%로 가장 많다"고 전했다. 러시아 의료진. [리아노보스티=연합뉴스 자료사진]    이밖에 모스크바 외곽 모스크바주에서 95명, 제2도시 상트페테르부르크에서 34명, 인근 레닌그라드주에서 32명의 신규 확진자가 보고됐다.     코로나19로 인한 사망자도 하루 사이 5명이 추가되면서 모두 63명으로 늘어났다.     정부 대책본부는 지금까지 확진